# VK API

VK API бесплатное, но нужно получить доступ


**Авторизация**

1. Перейти по ссылке https://vk.com/apps?act=manage
2. Создать новое приложение: standalone-приложение, название любое
3. Необходимо получить код подтверждения, может быть по номеру телефона
4. Если все ок, то вы перейдете в настройки приложения, там можно увидеть ключи доступа. На интересует **сервисный ключ доступа**

**Версии**

У API есть версии, поэтому вместе с токеном мы еще передаем информацию о том, какую версию мы используем. Версия отличает тем, какие параметры подаются, но больше это касается того, в каком виде мы получим ответ (структура ответа или наличие тех или иных полей может меняться от версии к версии).

In [ ]:
import requests
from tqdm.auto import tqdm
from datetime import datetime
import time
import pandas as pd

Зафиксируем версию и токен в константах. В качестве токена подставляется сервисный ключ из настроек приложения. Укажем ту, которая дефолтно показывается в документации.

In [ ]:
TOKEN = "5ddc69ee5ddc69ee5ddc69ee8f5eceded755ddc5ddc69ee3e2c0305f599d50fb0c3ca0c"
VERSION = "5.130"

## Метод wall.get

Первое, что мы сделаем - попробуем скачать информацию со стены, чтобы научиться собирать текстовые данные.

Подробная документация по ссылке: https://vk.com/dev/wall.get. Тут можно посмотреть на пример запроса, описание полей.

In [ ]:
wall_get_url = "https://api.vk.com/method/wall.get" # endpoint, на который мы отправляем такие запросы

Запрашиваем 2 последних поста со страницы юзера с id = 1 (Павел Дуров). 

Для сообществ ID будут отрицательными (например, -1)

In [ ]:
data = requests.get(
    wall_get_url, 
    params={
        "owner_id": 1,  # ID юзера
        "count": 2,  # кол-во постов
        "v": VERSION, # версия API
        "access_token": TOKEN  # токен доступа
    }
).json()

Мы получим ответ, который представляет собой словарь, где по ключу response лежит сам ответ.

Внутри лежит параметр count с числом записей (всего). В items сами посты (2, как мы просили).

Для каждого поста есть информация по объекту post. Подробное описание на странице https://vk.com/dev/objects/post

In [ ]:
type(data)

In [ ]:
data.keys()

In [ ]:
data['response'].keys()

In [ ]:
data['response']['count']

In [ ]:
type(data['response']['items'])

In [ ]:
len(data['response']['items'])

In [ ]:
data['response']['items'][0]

Можно заметить, что дата отображается в виде числа. Это специальный формат unixtimestamp, который очень часто используется, так как целые числа - это универсальный способ хранения, который можно исопльзовать в любой системе (JSON, любые БД и прочие)

In [ ]:
unixtime = data['response']['items'][0]['date']
utc = datetime.fromtimestamp(unixtime)
print(unixtime, utc)

## wall.getComments

Следующий метод - это сбор комментариев к посту, что тоже может быть интересно для изучения диалогов или чего-то подобного.

Основные параметры: owner_id, post_id. Их мы можем достать из информации о постах на стене.

Можно это сделать и вручную, например, если открыть пост во всплывающем окне, то по адресу в адресной строке можно понять эти id.

https://vk.com/id1?w=wall1_2442097 : owner_id = 1, post_id = 2442097

Для примера возьмем пост из СМИ, где можно оставлять комментарии.

In [ ]:
get_comments_url = "https://api.vk.com/method/wall.getComments"

In [ ]:
data = requests.get(
    get_comments_url, 
    params={
        "owner_id": -76982440,
        "post_id": 5011073,
        "count": 2,
        "need_likes": 1,
        "v": VERSION,
        "access_token": TOKEN
    }
).json()

In [ ]:
data

In [ ]:
{'response': {'count': 16,
  'items': [{'id': 5011074,
    'from_id': 432646319,
    'post_id': 5011073,
    'owner_id': -76982440,
    'parents_stack': [],
    'date': 1618533695,
    'text': 'Страна советов',
    'likes': {'count': 14, 'user_likes': 0, 'can_like': 1},
    'thread': {'count': 0,
     'items': [],
     'can_post': True,
     'show_reply_button': True,
     'groups_can_post': True}},
   {'id': 5011078,
    'from_id': 174352380,
    'post_id': 5011073,
    'owner_id': -76982440,
    'parents_stack': [],
    'date': 1618533953,
    'text': 'Красавчик 👍🏼❤️',
    'likes': {'count': 0, 'user_likes': 0, 'can_like': 1},
    'thread': {'count': 0,
     'items': [],
     'can_post': True,
     'show_reply_button': True,
     'groups_can_post': True}}],
  'current_level_count': 10,
  'can_post': True,
  'show_reply_button': True,
  'groups_can_post': True}}

## groups.getMembers

Можно получить информацию о пользователях, например, о тех, кто подписан на опредленную группу.

Попробуем еще параметр offset, который может пригодиться, когда данных много и придется скачивать "страницы" или блоки по 100 или 1000 объектов (1000 - ограничение для этого метода)

In [ ]:
group_members = "https://api.vk.com/method/groups.getMembers"

In [ ]:
group = "dormitory8hse"

In [ ]:
data = requests.get(
    group_members,
    params={
        'group_id': group,
        'access_token': TOKEN,
        'v': VERSION,
        'offset': 0
    }
).json()

In [ ]:
data["response"].keys()

Кол-во объектов всего

In [ ]:
data["response"]["count"]

В качестве ответа по людям - просто список ID. По ним уже дальше можно запрашивать подробную информацию о пользователях.

In [ ]:
len(data["response"]["items"])

In [ ]:
data["response"]["items"][:10]

А теперь следующая страница. Это можно делать в цикле, чтобы выкачать всех

In [ ]:
data = requests.get(
    group_members,
    params={
        'group_id': group,
        'access_token': TOKEN,
        'v': VERSION,
        'offset': 1000
    }
).json()

data["response"]["items"][:10]

## users.get

Попробуем собрать информацию о тех, кто попал

In [ ]:
users_get_url = "https://api.vk.com/method/users.get"

In [ ]:
all_users_data = []

for user in tqdm(data["response"]["items"][:500]):
    user_info = requests.get(
        users_get_url,
        params={
            'user_ids': user,
            'fields': 'bdate,home_town,universities',
            'access_token': TOKEN,
            'v': VERSION
        }
    ).json()
    all_users_data.extend(user_info["response"])
    time.sleep(0.2) # ОГРАНИЧЕНИЕ ПО ВРЕМЕНИ

In [ ]:
user_info

In [ ]:
df = pd.DataFrame(all_users_data)
# чтобы не показывать в открытом месте данные, возьмем для демонстрации толькодва столбца
# df = df.dropna(subset=["home_town", "universities"])
# df = df[["home_town", "universities"]] 

In [ ]:
df.head()

Университеты нужно еще дальше разбирать, а вот города можно подсчитать сразу

In [ ]:
df["home_town"].value_counts()

Видно, что не все идеально (есть сразу несколько), но после препроцессинга может быть ок.

## Задание

### 1

Попробовать собрать данные участников одной из групп и подсчитать статистику по какому-либо параметру (город, год рождения или иное).

1. Выбираем группу
2. Делаем пробный запрос
3. Пишем цикл по сбору (установка offset +1000 каждый раз)
4. Сохраняем ID
5. В цикле получаем информацию по пользователям
6. Сохраняем в датафрейм
7. Считаем статистику (и делаем препроцессинг, если нужно)

### 2

Собрать посты и/или комментарии, где авторы разные и попробовать сравнить употребление какого-либо слова у двух групп пользователей (Москва / не-Москва, Петербург / Москва, любые другие). Возможно, данных нужно будет много, но можно попробовать популярное слово. Главное - попробовать сделать основу для решения подобной задачи.

1. Выбираем группу
2. Собраем посты, сохраняем.
3. Для каждого поста собираем комментарии
4. По всем объектам собираем уникальные айди пользователей (лучше в порядке убывания частоты появления
5. Собираем информацию по ним, выделяем наши группы
6. Лемматизируем (если надо)
7. Считаем стастику

# Задача 1

In [ ]:
group = "dormitory8hse"
group_id_get_url = "https://api.vk.com/method/utils.resolveScreenName"
group_members = "https://api.vk.com/method/groups.getMembers"
wall_get_url = "https://api.vk.com/method/wall.get"
get_comments_url = "https://api.vk.com/method/wall.getComments"
users_get_url = "https://api.vk.com/method/users.get"

In [ ]:
group_id = requests.get(
    group_id_get_url,
    params={
        'screen_name': group,
        'access_token': TOKEN,
        'v': VERSION
    }
).json()
group_id

In [ ]:
all_data_posts = []

for i in tqdm(range(10)):
    data_posts = requests.get(
        wall_get_url,
        params={
            'owner_id': -group_id['response']['object_id'],
            'access_token': TOKEN,
            'v': VERSION,
            'count': 100,
            'offset': i*100
        }
    ).json()
    all_data_posts.extend(data_posts['response']['items'])
    time.sleep(0.1) # ОГРАНИЧЕНИЕ ПО ВРЕМЕНИ

In [ ]:
len(all_data_posts)

In [ ]:
all_data_posts[2]

In [ ]:
result = []
for post in tqdm(all_data_posts):
    data = requests.get(
        get_comments_url, 
        params={
            "owner_id": -group_id['response']['object_id'],
            "post_id": post['id'],
            "count": 100,
            "v": VERSION,
            "access_token": TOKEN
        }
    ).json()
    result.append(data)
    time.sleep(0.1) # ОГРАНИЧЕНИЕ ПО ВРЕМЕНИ

In [ ]:
result[3]

In [ ]:
post_comments = [x['response']['items'] for x in result if 'response' in x and x['response']['count'] > 0]

In [ ]:
post_comments = [comment for cooments in post_comments for comment in cooments]

In [ ]:
len(post_comments)

In [ ]:
post_comments

In [ ]:
uniq_users = list(set([comment['from_id'] for comment in post_comments if comment['from_id'] != 0]))
len(uniq_users), uniq_users[:10]

In [ ]:
user_result = []
for user in tqdm(uniq_users):
    data = requests.get(
        users_get_url, 
        params={
            'user_ids': user,
            'fields': 'activities,about,books,career,sex,bdate,interests,home_town,universities',
            'access_token': TOKEN,
            'v': VERSION
        }
    ).json()
    user_result.append(data)
    time.sleep(0.1) # ОГРАНИЧЕНИЕ ПО ВРЕМЕНИ

In [ ]:
user_result